In [1]:
import os

os.chdir("../")  # Укажите путь к корню проекта
print(os.getcwd())  # Проверьте, что путь изменился

c:\Users\MishaV\Desktop\karpov\ML\L22


In [2]:
import warnings
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from src.data import (
    select_data_csv,
    CustomTransformer,
    train_test_split,
    transform_data,
    OHE_COLS,
    MTE_COLS,
)
from src.train import save_model, load_model, model_score

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
# data = select_data(
#     seed=0.42,
#     count=4_800_000,
#     method="stratified",
#     users_scv=True,
#     posts_csv=True,
#     feed_data_csv=True,
#     save_path=r".\data\raw\stratified42.csv",
# )
data = pd.read_csv(r".\data\raw\stratified42.csv", parse_dates=["timestamp"])
data["timestamp"].is_monotonic_increasing

True

In [5]:
data.columns

Index(['timestamp', 'user_id', 'post_id', 'action', 'target', 'gender', 'age',
       'country', 'city', 'exp_group', 'os', 'source', 'text', 'topic'],
      dtype='object')

In [8]:
embeddings = pd.read_csv(r".\data\processed\table_posts.csv", index_col="post_id")
embeddings = embeddings[[f"bert_embedding_{i}" for i in range(10)]]

In [10]:
data = pd.merge(left=data, right=embeddings, left_on="post_id", right_index=True)

In [13]:
data.to_csv(r".\data\processed\stratified42_embeddings.csv")

In [14]:
print("target с значением 1: ", (data["target"] == 1).sum())
print("target с значением 0: ", (data["target"] == 0).sum())

target с значением 1:  589259
target с значением 0:  4369645


In [15]:
X = data.drop("target", axis=1)
y = data["target"]

# более новые данные (по timestamp) относим к тестовой выборке
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)
custom_cv = [(X_train.index, X_test.index)]

CatBoost

In [16]:
pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        (
            "model",
            CatBoostClassifier(
                random_state=42,
                auto_class_weights="Balanced",
                cat_features=OHE_COLS + MTE_COLS,
            ),
        ),
    ]
)

In [17]:
pipe.fit(X, y)

Learning rate set to 0.38982
0:	learn: 0.6810420	total: 2.42s	remaining: 40m 19s
1:	learn: 0.6753740	total: 4.48s	remaining: 37m 16s
2:	learn: 0.6730054	total: 6.55s	remaining: 36m 17s
3:	learn: 0.6720292	total: 8.62s	remaining: 35m 45s
4:	learn: 0.6715264	total: 10.6s	remaining: 35m 14s
5:	learn: 0.6643646	total: 12.7s	remaining: 34m 55s
6:	learn: 0.6627876	total: 14.5s	remaining: 34m 13s
7:	learn: 0.6619200	total: 16.9s	remaining: 34m 59s
8:	learn: 0.6608568	total: 19.1s	remaining: 35m 8s
9:	learn: 0.6607355	total: 21.4s	remaining: 35m 14s
10:	learn: 0.6604835	total: 23.4s	remaining: 35m 2s
11:	learn: 0.6600412	total: 25.1s	remaining: 34m 28s
12:	learn: 0.6598759	total: 27.2s	remaining: 34m 23s
13:	learn: 0.6593302	total: 29.7s	remaining: 34m 53s
14:	learn: 0.6586701	total: 31.6s	remaining: 34m 31s
15:	learn: 0.6578517	total: 33.3s	remaining: 34m 9s
16:	learn: 0.6576047	total: 35.2s	remaining: 33m 53s
17:	learn: 0.6572564	total: 37.4s	remaining: 33m 57s
18:	learn: 0.6568830	total: 39

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001BA4735C550>)])

In [18]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [19]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.6840433033875297
average_precision_score: 0.20311937982992326
hitrate: 0.4890007915063184


In [20]:
save_model("embeddings", pipe.named_steps["model"], method="catboost")

In [22]:
model = load_model("embeddings.cbm", method="catboost")

In [23]:
pipe.set_params(model=model)

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000001BA0BB8CDF0>)])

In [24]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [15]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.6205560268403599
average_precision_score: 0.17157966201713198
hitrate: 0.46403422582494064
